In [26]:
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from matplotlib import pylab as plt
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def get_data(filename):
    data_train = []
    data_test = []
    labels_train = []
    labels_test = []
    num_p_labels = 0
    num_n_labels = 0
    for line in open(filename):
        l = line[-2]
        d = process_line(line[:len(line)-3])
        is_positive = l == '1'
        is_negative = l == '0'
        if is_positive:
            num_p_labels += 1
            if num_p_labels <= 400:
                data_train.append(d)
                labels_train.append(l)
            else:
                data_test.append(d)
                labels_test.append(l)
        elif is_negative:
            num_n_labels += 1
            if num_n_labels <= 400:
                data_train.append(d)
                labels_train.append(l)
            else:
                data_test.append(d)
                labels_test.append(l)
        else:
            raise Exception('Bad label found')

    print('Positive labels: {0}'.format(num_p_labels))
    print('Negative laabels: {0}'.format(num_n_labels))
    return np.array(data_train), np.array(labels_train),\
           np.array(data_test), np.array(labels_test)

In [27]:
def process_line(line):
    l = line.lower()
    l = re.sub("[^a-z0-9]", "", l) # get rid of everything that isn't a letter or number
    l = lemmatizer.lemmatize(l)
    
    # get all of the words
    word_tokens = nltk.word_tokenize(l)
    ret_l = ''
    for word in word_tokens:
        if word not in stopwords: # get rid of stopwords
            ret_l += lemmatizer.lemmatize(word) + ' ' # lemmatize

    ret_l = ret_l[:len(ret_l)-2] # get rid of the tailing space
    return l

In [28]:
print('Loading data...')
am_trd, am_trl, am_ted, am_tld = get_data('./sentiment labelled sentences/amazon_cells_labelled.txt')
im_trd, im_trl, im_ted, im_tld = get_data('./sentiment labelled sentences/imdb_labelled.txt')
ye_trd, ye_trl, ye_ted, ye_tld = get_data('./sentiment labelled sentences/yelp_labelled.txt')

print('')
print('Training data: {0}'.format(len(am_trd) + len(im_trd) + len(ye_trd)))
print('Testing data: {0}'.format(len(am_ted) + len(im_ted) + len(ye_ted)))

Loading data...


LookupError: 
**********************************************************************
  Resource u'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\Ryan Farr/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Anaconda2\\nltk_data'
    - 'C:\\Anaconda2\\lib\\nltk_data'
    - 'C:\\Users\\Ryan Farr\\AppData\\Roaming\\nltk_data'
**********************************************************************

In [ ]:
features = {}
def gather_features(d1, d2, d3):
    for ds in (d1, d2, d3):
        for d in ds:
            sl = d.split(' ')
            for word in sl:
                if word not in features:
                    features[word] = len(features)
                
gather_features(am_trd, im_trd, ye_trd)
print('Number of features: {0}'.format(len(features)))

In [ ]:
def convert_bag_of_words(data):
    ret = []
    for d in data:
        bow = np.zeros(len(features))
        line = d.split(' ')
        for word in line:
            if word in features:
                bow[features[word]] += 1
        ret.append(bow)
    return np.array(ret)

amc_trd = convert_bag_of_words(am_trd)
amc_ted = convert_bag_of_words(am_ted)
imc_trd = convert_bag_of_words(im_trd)
imc_ted = convert_bag_of_words(im_ted)
yec_trd = convert_bag_of_words(ye_trd)
yec_ted = convert_bag_of_words(ye_ted)

In [ ]:
def postprocess(data):
    pass